In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import StructuredTool
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By


llm = ChatOpenAI(
    temperature=0.3,
    # model="gpt-4",
    )


def login_tool(login_url: str, id: str, pwd: str, btn_name: str):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_experimental_option("detach", True)
    
    global driver

    driver = webdriver.Chrome(options=chrome_options)
    driver.implicitly_wait(3)

    # 로그인 페이지로 이동
    driver.get(login_url)

    # 로그인 폼 요소 찾기
    id_input = driver.find_element(By.XPATH,'//*[@id="id"]')
    pwd_input = driver.find_element(By.XPATH, '//*[@id="password"]')

    # 사용자 이름과 비밀번호 입력
    id_input.send_keys(id)
    pwd_input.send_keys(pwd)

    element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, f"//button[contains(., '{btn_name}')]"))
    )
    element.click()
    return driver


# 마타와 XX 선택할 때
def first_tool(btn_name: str):
    if "마타와" in btn_name:
        cond = f"//a[.//h3[contains(., '{btn_name}')]]"
    elif "[마타와" in btn_name:
        cond = f"//a[contains(., '{btn_name}')]"
    else:
        cond = f"//button[contains(., '{btn_name}')]"
        
    element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, cond))
    )
    element.click()
    return driver


# STEP 01. 학년 선택하고 다음으로 이동
def second_tool(click_btn, move_btn):
    element = driver.find_elements(By.XPATH, f"//p[contains(., '{click_btn}')]")
    element[0].click()

    element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, f"//button[contains(., '{move_btn}')]"))
    )
    element.click()
    return driver


def third_tool(select_box_btn, grade_btn, selected_class_list, move_btn):
    placeholder = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, f"//div[contains(text(), '{select_box_btn}')]"))
    )
    placeholder.click()

    grade_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, f"//div[text()='{grade_btn}']"))
    )
    grade_element.click()

    # TODO: 복수 반 선택 로직
    for sc in selected_class_list:
        class_element = driver.find_element(By.XPATH, f"//div[contains(text(), '{sc}')]")
        class_element.click()
    
    move_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, f"//button[contains(., '{move_btn}')]"))
    )
    move_element.click()
    return driver


agent = initialize_agent(
    llm=llm, 
    verbose=True,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=[
        StructuredTool.from_function(
            func=login_tool,
            name="login tool",
            description="""
            With this tool, you can access a url where login is possible, enter the id and password(pwd), and by pressing a button, you can log in to the page.
            """,
        ), 
        StructuredTool.from_function(
            func=first_tool,
            name="first tool",
            description="""
            This tool is used simply for the action of clicking a single button.
            """,
        ),
        StructuredTool.from_function(
            func=second_tool,
            name="second tool",
            description="""
            This tool is used to select one condition and then move to the next page through a '다음' or '완료' button. A total of two clicks are involved.
            Use this tool in the final step of submission.
            """,
        ),
        StructuredTool.from_function(
            func=third_tool,
            name="third tool",
            description="""
            This tool allows you to first select a target through a placeholder, and then choose various conditions. Therefore, more than three clicks will occur.
            """,
        ),
    ],   
)


prompt = """
우리는 아래 단계대로 차례차례 진행해야돼. 
1. https://m5-dev.matamath.net/test.es/login 페이지로 접속해서 id: test, pwd: 2023ejrmffhfl!를 입력 후에 로그인 버튼을 클릭해서 로그인해줘. 
2. '마타와 개념학습' 버튼을 클릭해줘. 
3. 초3-2를 클릭하고 다음 버튼을 클릭해서 이동해줘. 
4. 10강을 선택하여 다음으로 이동해줘. 
5. 학습지 출제 대상을 선택하기 위해, "선택해주세요"를 먼저 눌러줘. 그리고 4학년을 선택하고, 1반만 선택하여 다음으로 이동해줘. 
6. 제출 마지막 단계에선 없음을 선택하고 완료를 클릭해서 이동해줘. 
7. 나의 수업 보기로 들어가줘. 
8. '마타와 개념학습' 버튼을 클릭해줘. 
9. 실시간 버튼을 클릭해줘. 
""" 
agent.invoke(prompt)



> Entering new AgentExecutor chain...
Thought: The user provided a series of steps that need to be followed in order to complete a task on a website. I will need to use the login tool, first tool, second tool, and third tool to complete the steps. I will provide the necessary JSON blobs for each step.

Action:
```
{
  "action": "login tool",
  "action_input": {
    "login_url": "https://m5-dev.matamath.net/test.es/login",
    "id": "test",
    "pwd": "2023ejrmffhfl!",
    "btn_name": "로그인"
  }
}
```



The chromedriver version (119.0.6045.105) detected in PATH at C:\Users\USER\AppData\Local\Programs\Python\Python311\chromedriver.exe might not be compatible with the detected chrome version (120.0.6099.131); currently, chromedriver 120.0.6099.109 is recommended for chrome 120.*, so it is advised to delete the driver in PATH and retry



Observation: <selenium.webdriver.chrome.webdriver.WebDriver (session="da6616d0d6d9157b242311a596af82a5")>
Thought:The login tool has been successfully executed and the user is now logged in. Now I need to click the '마타와 개념학습' button.

Action:
```
{
  "action": "first tool",
  "action_input": {
    "btn_name": "마타와 개념학습"
  }
}
```
Observation: <selenium.webdriver.chrome.webdriver.WebDriver (session="da6616d0d6d9157b242311a596af82a5")>
Thought:The '마타와 개념학습' button has been successfully clicked. Now I need to click on '초3-2' and then the '다음' button to proceed.

Action:
```
{
  "action": "second tool",
  "action_input": {
    "click_btn": "초3-2",
    "move_btn": "다음"
  }
}
```


Observation: <selenium.webdriver.chrome.webdriver.WebDriver (session="da6616d0d6d9157b242311a596af82a5")>
Thought:The '초3-2' button has been clicked and the '다음' button has been clicked as well. Now I need to select '10강' and click the '다음' button to proceed.

Action:
```
{
  "action": "second tool",
  "action_i

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF69DBD82B2+55298]
	(No symbol) [0x00007FF69DB45E02]
	(No symbol) [0x00007FF69DA005AB]
	(No symbol) [0x00007FF69DA4175C]
	(No symbol) [0x00007FF69DA418DC]
	(No symbol) [0x00007FF69DA7CBC7]
	(No symbol) [0x00007FF69DA620EF]
	(No symbol) [0x00007FF69DA7AAA4]
	(No symbol) [0x00007FF69DA61E83]
	(No symbol) [0x00007FF69DA3670A]
	(No symbol) [0x00007FF69DA37964]
	GetHandleVerifier [0x00007FF69DF50AAB+3694587]
	GetHandleVerifier [0x00007FF69DFA728E+4048862]
	GetHandleVerifier [0x00007FF69DF9F173+4015811]
	GetHandleVerifier [0x00007FF69DC747D6+695590]
	(No symbol) [0x00007FF69DB50CE8]
	(No symbol) [0x00007FF69DB4CF34]
	(No symbol) [0x00007FF69DB4D062]
	(No symbol) [0x00007FF69DB3D3A3]
	BaseThreadInitThunk [0x00007FFB1DD9257D+29]
	RtlUserThreadStart [0x00007FFB1F2CAA58+40]
